In [1]:
import selenium
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException
import re
from datetime import datetime
from time import sleep
from tqdm import tqdm_notebook
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 1. 더마토리 캠페인 인플루언서

In [2]:
der_inf = pd.read_excel('더마토리_캠페인인플루언서목록.xlsx')
der_inf.head()

,블로거,오픈 URL
0,우한경,https://www.instagram.com/p/B2J6LtSnJ90/
1,아름송이,https://www.instagram.com/p/B1-vy77AsTs/
2,조효진,https://www.instagram.com/p/B18cgySnIe7/
3,공지민,https://www.instagram.com/p/B13hxWaAnlZ/
4,NaN,https://www.instagram.com/stories/velyvely0_/


## 인스타 로그인

In [54]:
# driver path
path = 'C:/Users/Administrator/chromedriver'
browser = webdriver.Chrome(path)

In [55]:
# 인스타 접속
url = 'https://www.instagram.com/?hl=ko'
browser.get(url)
time.sleep(3)

# 인스타 로그인 페이지 접속
login = browser.find_element_by_css_selector('#react-root > section > main > article > div.rgFsT > div:nth-child(2) > p > a')
browser.get(login.get_attribute('href'))

# 인스타 아이디 비번 입력
cus_id = 'tnwlsl0612@naver.com'
cus_pw = '98520wls!'

time.sleep(3)
id_input = browser.find_element_by_css_selector("#react-root > section > main > div > article > div > div:nth-child(1) > div > form > div:nth-child(2) > div > label > input")
id_input.send_keys(cus_id)

pw_input = browser.find_element_by_css_selector("#react-root > section > main > div > article > div > div:nth-child(1) > div > form > div:nth-child(3) > div > label > input")
pw_input.send_keys(cus_pw)

login_click = browser.find_element_by_css_selector("#react-root > section > main > div > article > div > div:nth-child(1) > div > form > div.Igw0E.IwRSH.eGOV_._4EzTm.bkEs3.CovQj.jKUp7.DhRcB > button > div")
login_click.click()

# 알림 설정 창 -> 나중에 알리기 클릭
time.sleep(2)
later = browser.find_element_by_css_selector("body > div.RnEpo.Yx5HN > div > div > div.mt3GC > button.aOOlW.HoLwm")
later.click()

## 댓글 긁기

In [56]:
# 에러날 경우 그대로 ctrl+enter 치면 돌아가게끔
start_idx = 46
end_idx = der_inf.shape[0]

In [57]:
start_idx

46

In [58]:
der_inf.iloc[46, :]

블로거                                            구사자
오픈 URL    https://www.instagram.com/p/B2yNGdCn733/
Name: 46, dtype: object

In [51]:
(now_reply-start_reply).seconds

240

In [61]:
initial_idx = start_idx
reply_list = []
url_list = []
for idx, url in enumerate(tqdm_notebook(der_inf['오픈 URL'].iloc[start_idx:end_idx])) :
    whole_idx = initial_idx + idx
    browser.get(url)
    time.sleep(1)
    
    ## 댓글 더보기 버튼이 있는 경우 ##
    if 'Load more comments' in browser.page_source:
        start_reply = datetime.now()
        while True : # 댓글 더보기 버튼 클릭(없을 때까지)                       
            try :
                btn = browser.find_element_by_css_selector("span.glyphsSpriteCircle_add__outline__24__grey_9.u-__7")
                btn.click() #버튼클릭
                now_reply = datetime.now()
                time.sleep(1)
                if (now_reply-start_reply).seconds > 120:
                    break
                else:
                    continue
            except NoSuchElementException:
                break
                
        # 만약 대댓이 있다면 클릭해서 다 여는 과정 
        if "답글 보기(1개)" in browser.page_source :
            try: # 댓글 보기 추출
                more_more_btn = browser.find_elements_by_css_selector('div.Igw0E.IwRSH.eGOV_.ybXk5._4EzTm')    
                # 댓글 보기 전부 클릭 
                for bnt_click in more_more_btn : 
                    time.sleep(1)
                    try:
                        bnt_click.click() 
                        time.sleep(0.5)
                    except StaleElementReferenceException:
                        break
                    time.sleep(1.5) 
            except StaleElementReferenceException:
                break

        # 댓글 추출
        replys =  browser.find_elements_by_css_selector('div.C4VMK')
        cus_name_list = []
        cus_content_list = []
        for reply in replys:
            splits = reply.text.split('\n')
            cus_name = splits[0]
            cus_content = ','.join(splits[1:])
            reply_list.append((der_inf['블로거'][start_idx],cus_name, cus_content))
            
        # url 추출
        urls = browser.find_elements_by_css_selector('a.FPmhX.notranslate.TlrDj')      
        for url in urls:
            cus_url = url.get_attribute('href')
            url_list.append((der_inf['블로거'][start_idx], cus_url))

    ############################################################################
    ## 댓글 더보기가 없는 경우 ##
    else:
        # 만약 대댓이 있다면 클릭해서 다 여는 과정 
        if "답글 보기(1개)" in browser.page_source : 
            # 댓글 보기 추출 
            more_more_btn =  browser.find_elements_by_css_selector('div.Igw0E.IwRSH.eGOV_.ybXk5._4EzTm')   
            # 댓글 보기 전부 클릭 
            for bnt_click in more_more_btn : 
                bnt_click.click() 
                time.sleep(0.5)

        # 댓글 추출
        replys =  browser.find_elements_by_css_selector('div.C4VMK')
        cus_name_list = []
        cus_content_list = []
        for reply in replys:
            splits = reply.text.split('\n')
            cus_name = splits[0]
            cus_content = ','.join(splits[1:])
            reply_list.append((der_inf['블로거'][start_idx],cus_name, cus_content))
        
        # url 추출
        urls = browser.find_elements_by_css_selector('a.FPmhX.notranslate.TlrDj')      
        for url in urls:
            cus_url = url.get_attribute('href')
            url_list.append((der_inf['블로거'][start_idx], cus_url))
    

    start_idx += 1
    time.sleep(2)

In [62]:
reply_list

[('구사자',
  '94_j.a',
  '⠀,♥️ 오랜만에 #댓글이벤트 ♥️,⠀,피부 진정효과로 유명한 #더마진정패드 ♥,닦토로도 좋지만 팩처럼 붙이고 있으면,더 집중적인 피부 진정케어를 할 수 있어 요즘같은 환절기에 최고 😊,⠀,시카 에센스가 듬뿍 들어가 있어,빠르고 간편하게 피부 진정효과를 느낄 수 있어요,저는 요즘 매일 저녁 사용하고 있는데 집게로 한 장 떼내서 붙이기만 하면 되니까,위생적이고 간편해서 자주 손이 가는 것 같아요 ㅎㅎ,⠀ ⠀,환절기 피부관리로 스트레스 받으시는 분들 #댓글 남겨주세요! ⠀,✌🏻댓글 남겨주시는 5분 선정해서 #더마토리 #붙이는진정패드 나눠드려요 ! ⠀ ⠀,,#진정패드 #민감피부 #스킨케어 #패드추천 #AD,2주'),
 ('구사자', 'ro.junghwa', '마지막사진 쥔심 개예빠,2주좋아요 4개답글 달기'),
 ('구사자', '94_j.a', '@ro.junghwa ㅋㅋㅋㅋㅋㅋㅋ ☺️,2주좋아요 2개답글 달기'),
 ('구사자',
  'see.u_u971209',
  '피부가 갑자기 건조 해지더니 뒤집어져서 피부과 다니고 고생중입니다 흐아ㅠ,진정효과가 있다니 써보고 싶네여😆,2주좋아요 1개답글 달기'),
 ('구사자', 'ji.hye95', '원래 피부가 예민한데 요즘들어 자꾸 이미에 뾰루지가 나서 고민이예요 ㅜㅠ,2주답글 달기'),
 ('구사자',
  '_soo_young_0930',
  '눈동자가 진짜 너무 특별하시고 이쁘세요 ㅠㅠ 투명 렌즈만 끼셨나요?? 언니!! 진짜 눈 너무 이뻐서 가지고 싶어요 ㅋㅋㅋ🥺너무 너무 이뻐요 ㅠㅠ진쨔 ㅠㅠ,2주답글 달기'),
 ('구사자', 's_ash01', '환절기라 더 건조해지고 그런데...마침 이런 이벤트를ㅜㅜㅜ당신은 천사,2주답글 달기'),
 ('구사자',
  'hanac_jon',
  '저여 ㅜㅜㅜ저 피부도 아토피때문에 피부도 민감한데 사춘기때라 얼굴애 여드름도 많이 나고 피부도 많이 뒤집어지더라구여 ㅠㅠㅠ화장에 관심이 많아서 언니차럼 예쁜 사람들보구 화장따라하기도 하고 콤

In [63]:
reply_df = pd.DataFrame(reply_list, columns = ['인플루언서', '댓글user', '댓글내용'])
reply_df.to_excel('더마토리_캠페인인플루언서댓글(구사자_정이한).xlsx', index = 0)

## 합치기

In [129]:
reply_df1 = pd.read_excel('더마토리_캠페인인플루언서댓글(0_워뇽).xlsx')
reply_df2 = pd.read_excel('더마토리_캠페인인플루언서댓글(김나희_김이화).xlsx')
reply_df3 = pd.read_excel('더마토리_캠페인인플루언서댓글(구사자_정이한).xlsx')

In [151]:
reply = pd.concat([reply_df1,reply_df2,reply_df3], axis=0).reset_index()
reply = reply.iloc[:,1:]
reply.head()

,인플루언서,댓글user,댓글내용
0,우한경,han_kyung__,"요즘 놀러다니면서 피부도 다 타고 자극받고 예민해졌었는데,그거때문에 진정제품 알아보..."
1,우한경,taka12832019,"VeryGood👍🎶VeryＣute可愛い(^^)/よ🌹❤❤❤💕💕💕💕💕,4주답글 달기"
2,우한경,luv.ly_jin,우연히 에비앙에서 찍은 사진을 봤는데 진짜 예쁘셔서 선팔했어요❤️ 매번 입으시는 옷...
3,우한경,22.21pm,"매우 예쁘다 !!,4주답글 달기"
4,우한경,kappe408,"😍💕,4주답글 달기"


In [152]:
reply['댓글내용'][0]

'요즘 놀러다니면서 피부도 다 타고 자극받고 예민해졌었는데,그거때문에 진정제품 알아보다가 찾은 꿀템! 알려드릴게용,,유튜버들도 진짜 많이 쓰길래 저도 한 번 써봤는데,피부 예민했던 부위에 빠르고 간편하게 진정효과를 느낄 수 있었던 #더마토리 #붙이는진정패드 !!!!! 이번 여름에 물놀이 많이 다녔는데,햇빛이나 수영장물로 피부가 민감해질 때마다 패드를 올려주면,진짜 시원해서 넘 좋구 잔뜩 자극받은 피부가 빠르게 진정되어서 신기했어요>_<ㅎㅎ,잠깐 선베드 누워서 쉴 때 틈틈히 붙여줘도 좋고 수영 다 끝나구 저녁에 붙여줘도 굳굳!,아 그리고 패드 한 장에도 시카 에센스가 완전 듬뿍 적셔있어서,저는 주로 팩으로 사용하는데 더 효과가 좋았던 거 같아요!!!! 알아보니 피부 진정효과로 유명한 제품이고,,쓸때마다 좋아서 지인들한테 선물해줬는데 다 너무 만족하더라구요>< 댓글을 남겨주시면 5분께 저의 진짜 인생 진정템을 보내드릴게요!,#더마토리 #더마진정패드 #붙이는진정패드 #AD #시카진정패드 #강력진정패드 #임상완료진정패드 #랄라블라1위패드 #화해1위패드 #멸균진정패드 #빠른진정패드 #진정패드 #패드추천,#스킨케어 #피부관리,4주'

In [153]:
reply['댓글내용'] = reply['댓글내용'].apply(lambda x: x.split(',')[0:-1])
reply['댓글내용'][0]

['요즘 놀러다니면서 피부도 다 타고 자극받고 예민해졌었는데',
 '그거때문에 진정제품 알아보다가 찾은 꿀템! 알려드릴게용',
 '',
 '유튜버들도 진짜 많이 쓰길래 저도 한 번 써봤는데',
 '피부 예민했던 부위에 빠르고 간편하게 진정효과를 느낄 수 있었던 #더마토리 #붙이는진정패드 !!!!! 이번 여름에 물놀이 많이 다녔는데',
 '햇빛이나 수영장물로 피부가 민감해질 때마다 패드를 올려주면',
 '진짜 시원해서 넘 좋구 잔뜩 자극받은 피부가 빠르게 진정되어서 신기했어요>_<ㅎㅎ',
 '잠깐 선베드 누워서 쉴 때 틈틈히 붙여줘도 좋고 수영 다 끝나구 저녁에 붙여줘도 굳굳!',
 '아 그리고 패드 한 장에도 시카 에센스가 완전 듬뿍 적셔있어서',
 '저는 주로 팩으로 사용하는데 더 효과가 좋았던 거 같아요!!!! 알아보니 피부 진정효과로 유명한 제품이고',
 '',
 '쓸때마다 좋아서 지인들한테 선물해줬는데 다 너무 만족하더라구요>< 댓글을 남겨주시면 5분께 저의 진짜 인생 진정템을 보내드릴게요!',
 '#더마토리 #더마진정패드 #붙이는진정패드 #AD #시카진정패드 #강력진정패드 #임상완료진정패드 #랄라블라1위패드 #화해1위패드 #멸균진정패드 #빠른진정패드 #진정패드 #패드추천',
 '#스킨케어 #피부관리']

In [154]:
reply['댓글내용'] = reply['댓글내용'].apply(lambda x: ','.join(x))
reply['댓글내용'][0]

'요즘 놀러다니면서 피부도 다 타고 자극받고 예민해졌었는데,그거때문에 진정제품 알아보다가 찾은 꿀템! 알려드릴게용,,유튜버들도 진짜 많이 쓰길래 저도 한 번 써봤는데,피부 예민했던 부위에 빠르고 간편하게 진정효과를 느낄 수 있었던 #더마토리 #붙이는진정패드 !!!!! 이번 여름에 물놀이 많이 다녔는데,햇빛이나 수영장물로 피부가 민감해질 때마다 패드를 올려주면,진짜 시원해서 넘 좋구 잔뜩 자극받은 피부가 빠르게 진정되어서 신기했어요>_<ㅎㅎ,잠깐 선베드 누워서 쉴 때 틈틈히 붙여줘도 좋고 수영 다 끝나구 저녁에 붙여줘도 굳굳!,아 그리고 패드 한 장에도 시카 에센스가 완전 듬뿍 적셔있어서,저는 주로 팩으로 사용하는데 더 효과가 좋았던 거 같아요!!!! 알아보니 피부 진정효과로 유명한 제품이고,,쓸때마다 좋아서 지인들한테 선물해줬는데 다 너무 만족하더라구요>< 댓글을 남겨주시면 5분께 저의 진짜 인생 진정템을 보내드릴게요!,#더마토리 #더마진정패드 #붙이는진정패드 #AD #시카진정패드 #강력진정패드 #임상완료진정패드 #랄라블라1위패드 #화해1위패드 #멸균진정패드 #빠른진정패드 #진정패드 #패드추천,#스킨케어 #피부관리'

## word-count

In [158]:
from sklearn.feature_extraction.text import CountVectorizer
import re
from worin.tag import Worin

tagger = Worin()   

def get_nouns(text):
    nouns = tagger.nouns(text)
    nouns = [word for word in nouns if len(word) > 1]
    return nouns

cv = CountVectorizer(max_features=1000, tokenizer=get_nouns)
tdm = cv.fit_transform(reply['댓글내용'])
cv.get_feature_names()[:10]

['가능', '가을', '가족', '가지', '각질', '각질제', '간단', '간절', '간지', '간편']

In [159]:
count = tdm.sum(axis=0)
word_count = pd.DataFrame({
    '단어': cv.get_feature_names(),
    '빈도': count.flat})

C:\Users\Administrator\Anaconda3\lib\site-packages\pandas\core\dtypes\dtypes.py:458: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


In [160]:
sorted_df = word_count.sort_values('빈도', ascending=False)  # 내림차순 정렬
sorted_df.tail()

,단어,빈도
545,여행가서,3
546,여행갈때,3
631,이리,3
226,렌징,3
399,세번,3


In [162]:
sorted_df.to_excel('더마토리 인스타그램 게시글 단어 빈도.xlsx')

In [165]:
sorted_df.head(10)

,단어,빈도
904,피부,1936
778,진정,1004
878,패드,802
580,요즘,550
361,사용,453
979,환절기,452
512,언니,450
514,얼굴,366
731,제품,362
924,필요,345


##  tf-idf 유사도 

In [234]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
tdm2 = tfidf.fit_transform(tdm)
words = cv.get_feature_names()

for i, n in sorted(zip(tdm2[0].indices, tdm2[0].data)):
    print(words[i], n)

간편 0.11545206601408754
강력진정패드 0.16082544058951545
꿀템 0.15635532059401858
댓글 0.11545206601408754
더마진정패드 0.1110701675131417
더마토리 0.1821747285134882
랄라블라 0.16642369522469327
멸균진정패드 0.16642369522469327
민감 0.091745813128108
부위 0.12874009252531737
사용 0.08141572136370114
선물 0.12400509547009422
스킨케어 0.12353611186881717
시원 0.13554274308060998
시카 0.10567913203046646
시카진정패드 0.1244841655095142
신기 0.1698575400849421
에센스 0.11359466577676464
여름 0.12704743819242778
예민 0.1661762145806178
요즘 0.0711129614523205
위패드 0.30196643830530323
유명 0.1209123889500055
인생 0.1430379643902592
임상완료진정패드 0.1698575400849421
자극 0.22659923290655146
저녁 0.13400586489483254
정패드 0.2786652520056234
제품 0.08401319101104963
진정 0.06257144210590067
진정제품 0.1494463409668742
진정템 0.15847844046230083
진정패드 0.09616648604834449
진정효과 0.24265754600642217
진짜 0.23566993213136972
추천 0.09348247649393189
패드 0.20385523130277425
피부 0.14994390684895023
피부관리 0.12704743819242778
햇빛 0.14538496451747382
화해 0.14091484452197692
효과 0.10083367209760025


In [238]:
tdm2.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [239]:
tdm2.indices

array([985, 973, 943, ...,  71, 988, 567], dtype=int32)

In [246]:
len(tdm2.indices)

21125

In [245]:
cv.get_feature_names()[0]

'가능'

In [250]:
tfidf_df = pd.DataFrame(tdm2.toarray(), columns=cv.get_feature_names())
tfidf_df.head()

,가능,가을,가족,가지,각질,각질제,간단,간절,간지,간편,...,후기,후회,휴가,흉터,흑흑,흡수,흥둥,흥둥이,흥둥이홈트,힐링
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.115452,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [258]:
from operator import itemgetter

def top10(labels):
    freq_words = []
    for i in range(10):
        count = tdm2[labels == i, :].sum(axis=0)
        ws = [w for w, n in sorted(zip(words, count.flat), key=itemgetter(1), reverse=True)[:15]]
        freq_words.append(ws)

    return pd.DataFrame(freq_words)

In [252]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=10, random_state=1234)
labels_km = km.fit_predict(tdm2)

In [259]:
top10(labels_km)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,저요저,얼굴,사용,신청,당첨,제품,피부,이벤트,저용,요즘,감사,진정패드,스트레스,화장,패드
1,이거,사용,공구,사보,피부,요즘,진정,사라,조아,사야,화장,아이템,당첨자,면접,과제
2,나야,사람,더마토리,흥둥이,흥둥이홈트,홈트,개월,피부,꿀피부,도움,이벤트,귀욤,마토리,요즘,운동끝
3,누나,피부,사진,사랑,칭찬,항상,선물,고마,기회,관리,복학,댓글,군대,원합,처음
4,관리,피부,민감,요즘,환절기,패드,이벤트,감사,진정,사용,건강,정패드,운동,필요,스트레스
5,저욥,얼굴,가능,가을,가족,가지,각질,각질제,간단,간절,간지,간편,감기,감사,감사드
6,언니,피부,추천,사랑,패드,요즘,진정,제품,이벤트,꿀피부,화장,환절기,당첨,이거,민감
7,저요,피부,요즘,언니,예민,환절기,건조,진정,스트레스,시작,패드,필요,정패드,얼굴,시퍼
8,진정,패드,필요,피부,얼굴,사용,제품,환절기,요즘,정패드,여드름,운동,마음,예민,화장
9,피부,환절기,요즘,예민,건조,진정,스트레스,민감,사용,화장,패드,고민,제품,여드름,때문


## word2vec

In [189]:
## Worin 형태소 분석기 사용
tagger= Worin()
def get_nouns(text):
    # 에러가 발생하지 않을 경우
    nouns = tagger.nouns(text)
    nouns = [word for word in nouns if len(word) > 1]
    return nouns

In [193]:
documents = []
for i in range(len(reply['댓글내용'])):
    documents.append(get_nouns(reply['댓글내용'][i]))

In [194]:
documents[0]

['요즘',
 '피부',
 '자극',
 '예민',
 '진정제품',
 '꿀템',
 '튜버',
 '예민',
 '부위',
 '간편',
 '진정효과',
 '더마토리',
 '정패드',
 '여름',
 '햇빛',
 '수영장물',
 '민감',
 '패드',
 '진짜',
 '시원',
 '자극',
 '피부',
 '진정',
 '신기',
 '선베드',
 '수영',
 '저녁',
 '패드',
 '시카',
 '에센스',
 '사용',
 '효과',
 '피부',
 '진정효과',
 '유명',
 '제품',
 '쓸때마',
 '선물',
 '만족',
 '댓글',
 '진짜',
 '인생',
 '진정템',
 '더마토리',
 '더마진정패드',
 '정패드',
 '시카진정패드',
 '강력진정패드',
 '임상완료진정패드',
 '랄라블라',
 '위패드',
 '화해',
 '위패드',
 '멸균진정패드',
 '정패드',
 '진정패드',
 '패드',
 '추천',
 '스킨케어',
 '피부관리']

In [230]:
from gensim.models.word2vec import Word2Vec
model = Word2Vec(documents,         # 리스트 형태의 데이터
                 sg=1,         # 0: CBOW, 1: Skip-gram
                 size=150,     # 벡터 크기
                 window=3,     # 고려할 앞뒤 폭(앞뒤 3단어)
                 min_count=3,  # 사용할 단어의 최소 빈도(3회 이하 단어 무시)
                 workers=4)    # 동시에 처리할 작업 수(코어 수와 비슷하게 설정)

C:\Users\Administrator\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [231]:
model.wv.vocab

{'요즘': <gensim.models.keyedvectors.Vocab at 0x25509643cf8>,
 '피부': <gensim.models.keyedvectors.Vocab at 0x2550965cac8>,
 '자극': <gensim.models.keyedvectors.Vocab at 0x2550965ce10>,
 '예민': <gensim.models.keyedvectors.Vocab at 0x2550965ca58>,
 '진정제품': <gensim.models.keyedvectors.Vocab at 0x2550965ca90>,
 '꿀템': <gensim.models.keyedvectors.Vocab at 0x2550965ce80>,
 '부위': <gensim.models.keyedvectors.Vocab at 0x2550965cd30>,
 '간편': <gensim.models.keyedvectors.Vocab at 0x2550965ccf8>,
 '진정효과': <gensim.models.keyedvectors.Vocab at 0x2550965ccc0>,
 '더마토리': <gensim.models.keyedvectors.Vocab at 0x2550965cc88>,
 '정패드': <gensim.models.keyedvectors.Vocab at 0x2550965cc50>,
 '여름': <gensim.models.keyedvectors.Vocab at 0x2550965cc18>,
 '햇빛': <gensim.models.keyedvectors.Vocab at 0x2550965cbe0>,
 '민감': <gensim.models.keyedvectors.Vocab at 0x2550965cba8>,
 '패드': <gensim.models.keyedvectors.Vocab at 0x2550965cb00>,
 '진짜': <gensim.models.keyedvectors.Vocab at 0x2550965c7b8>,
 '시원': <gensim.models.keyedvector

In [260]:
model.wv.most_similar('환절기')

[('민감', 0.9995136260986328),
 ('뒤집', 0.9994797706604004),
 ('예민', 0.9994595050811768),
 ('홍조', 0.9994474649429321),
 ('피부', 0.9994458556175232),
 ('걱정', 0.9994124174118042),
 ('요즘', 0.9994121789932251),
 ('난리', 0.9994069933891296),
 ('트러블', 0.9994018077850342),
 ('건조', 0.999401330947876)]

In [261]:
from gensim.models.word2vec import Word2Vec
model = Word2Vec(documents,         # 리스트 형태의 데이터
                 sg=1,         # 0: CBOW, 1: Skip-gram
                 size=100,     # 벡터 크기
                 window=3,     # 고려할 앞뒤 폭(앞뒤 3단어)
                 min_count=2,  # 사용할 단어의 최소 빈도(3회 이하 단어 무시)
                 workers=4)    # 동시에 처리할 작업 수(코어 수와 비슷하게 설정)

C:\Users\Administrator\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [262]:
model.wv.vocab

{'요즘': <gensim.models.keyedvectors.Vocab at 0x255071d6c50>,
 '피부': <gensim.models.keyedvectors.Vocab at 0x255072162e8>,
 '자극': <gensim.models.keyedvectors.Vocab at 0x25507212e48>,
 '예민': <gensim.models.keyedvectors.Vocab at 0x25507212ba8>,
 '진정제품': <gensim.models.keyedvectors.Vocab at 0x25507212e80>,
 '꿀템': <gensim.models.keyedvectors.Vocab at 0x25507212320>,
 '튜버': <gensim.models.keyedvectors.Vocab at 0x25507212ef0>,
 '부위': <gensim.models.keyedvectors.Vocab at 0x25507212710>,
 '간편': <gensim.models.keyedvectors.Vocab at 0x25507212128>,
 '진정효과': <gensim.models.keyedvectors.Vocab at 0x255072120b8>,
 '더마토리': <gensim.models.keyedvectors.Vocab at 0x255072122b0>,
 '정패드': <gensim.models.keyedvectors.Vocab at 0x25507212828>,
 '여름': <gensim.models.keyedvectors.Vocab at 0x25507212d30>,
 '햇빛': <gensim.models.keyedvectors.Vocab at 0x25507212550>,
 '수영장물': <gensim.models.keyedvectors.Vocab at 0x255072123c8>,
 '민감': <gensim.models.keyedvectors.Vocab at 0x25507212940>,
 '패드': <gensim.models.keyedvect

In [268]:
model.wv.most_similar('스트레칭')

[('아이', 0.9996206760406494),
 ('친구', 0.9996158480644226),
 ('예전', 0.999613344669342),
 ('방법', 0.9996033906936646),
 ('최고', 0.9995855689048767),
 ('저두', 0.9995819926261902),
 ('클렌징', 0.9995813369750977),
 ('홀리언니', 0.9995802640914917),
 ('학교', 0.9995771646499634),
 ('퇴근', 0.9995747208595276)]

# 2.더마토리 체험단

In [270]:
event = pd.read_excel('더마토리 하이포알러제닉 시카 레스큐 거즈 패드 인스타그램 체험단 응모하기(응답).xlsx')
event.head()

,타임스탬프,이름을 입력해주세요,연락처를 입력해주세요,주소를 입력해주세요,리뷰할 인스타그램 계정을 입력해주세요,더마토리 진정 보습 패드에 대한 기대평을 기입해주세요,아래 항목을 체크해주세요,수집한 개인정보의 위탁 및 마케팅 활용에 동의하십니까?
0,2019-08-22 19:40:41.980,김예진,01087624505,광주광역시 광산구 월계로 170 부영 1차 110동 801호,beauty_s_gm,닦아내는 토너로 깔끔하게 피부결 정리하고 싶어서 신청합니당,예,예
1,2019-08-22 20:05:56.481,김미나,01051236566,전라북도 전주시 완산구 효자동3가\n서곡로11(대림아파트) 105동 1002호,mimi95s2,"브링그린,어퓨,자연을 담은 유리병의 패드를 써봤는데 패드가 너무 편하기도 하고 좋더...",예,예
2,2019-08-22 20:08:24.951,이수민,01064096911,충남공주시신관동대아곰나루아파트102동1004호,4su_to_5su,피부로 바로 흡수될 것 같은 촉촉한 그느낌!!,예,예
3,2019-08-22 20:09:54.546,강혜영,01040632912,창원시 의창구 감계로221 감계 푸르지오 108동 302호,https://www.instagram.com/p/B1dmq9JAJV0/?igshi...,진짜 피부트러블에 좋을거같아요 울긋불긋 시카패드님 제피부를 잡아주세요,예,예
4,2019-08-22 20:14:06.122,정미현,010-2550-3271,경기도 안양시 만안구 병목안로61 성원2차아파트 202동 904호,mihyun_98,평소 민감성피부를 가지고 있어서 시카패드에 대한 기대가 높아요! 예민한피부에도 효과...,예,예


In [271]:
testi = event.iloc[:,5]
testi.head()

0                     닦아내는 토너로 깔끔하게 피부결 정리하고 싶어서 신청합니당
1    브링그린,어퓨,자연을 담은 유리병의 패드를 써봤는데 패드가 너무 편하기도 하고 좋더...
2                            피부로 바로 흡수될 것 같은 촉촉한 그느낌!!
3              진짜 피부트러블에 좋을거같아요 울긋불긋 시카패드님 제피부를 잡아주세요 
4    평소 민감성피부를 가지고 있어서 시카패드에 대한 기대가 높아요! 예민한피부에도 효과...
Name: 더마토리 진정 보습 패드에 대한 기대평을 기입해주세요, dtype: object

In [273]:
from sklearn.feature_extraction.text import CountVectorizer
import re
from worin.tag import Worin

tagger = Worin()   

def get_nouns(text):
    nouns = tagger.nouns(text)
    nouns = [word for word in nouns if len(word) > 1]
    return nouns

cv1 = CountVectorizer(max_features=1000, tokenizer=get_nouns)
tdm1 = cv1.fit_transform(testi)
cv1.get_feature_names()[:10]

['가격', '가능', '가득', '가볍', '가성비', '가을', '가지', '각질', '각질관리', '각질제']

In [274]:
count1 = tdm1.sum(axis=0)
word_count1 = pd.DataFrame({
    '단어': cv1.get_feature_names(),
    '빈도': count1.flat})

C:\Users\Administrator\Anaconda3\lib\site-packages\pandas\core\dtypes\dtypes.py:458: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


In [277]:
sorted_df1 = word_count1.sort_values('빈도', ascending=False)  # 내림차순 정렬
sorted_df1.tail()

,단어,빈도
888,폭탄,1
829,패드늗,1
885,포포크림,1
658,저번시리즈,1
876,편함,1


In [278]:
sorted_df1.to_excel('더마토리_체험단댓글 단어 빈도.xlsx')